# Ordinal Neural Network

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
from scipy.stats import pearsonr, kendalltau
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler


In [2]:
project_directory = r'C:\Users\marco\OneDrive\Desktop\Final Year Project'
os.chdir(project_directory)
base_dir = os.getcwd() 

In [5]:
class Ordinal(tf.keras.Model):
    def __init__(self, num_classes):
        super(Ordinal, self).__init__()
        self.num_classes = num_classes
        self.output_layer = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, inputs):
        outputs = self.output_layer(inputs)
        probs = tf.concat([
            outputs[..., :1],
            outputs[..., 1:] - outputs[..., :-1],
            1 - outputs[..., -1:]
        ], axis=-1)
        return outputs, probs

def create_ordinal_model(num_classes, input_dim):
    input_layer = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(64, activation='relu')(input_layer)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    outputs, probabilities = Ordinal(num_classes)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=[outputs, probabilities])
    model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy', None])
    return model

def concordance_correlation_coefficient(y_true, y_pred):
    cor = np.corrcoef(y_true, y_pred)[0][1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    return numerator / denominator

def pearson_correlation_coefficient(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

def kendalls_tau_coefficient(y_true, y_pred):
    return kendalltau(y_true, y_pred)[0]

def evaluate_individual_performance(X_test, Y_test, group_labels, ord_y_pred_score, target):
    evaluation_results = []
    for participant_id in np.unique(group_labels):
        idx = group_labels == participant_id
        participant_labels = Y_test[idx]
        participant_predictions = np.argmax(ord_y_pred_score[idx], axis=1)

        pcc_value = pearsonr(participant_labels, participant_predictions)[0]
        ccc_value = concordance_correlation_coefficient(participant_labels, participant_predictions)
        kendall_tau_value = kendalltau(participant_labels, participant_predictions)[0]

        evaluation_results.append({
            'Target': target,
            'Participant ID': participant_id,
            'PCC': pcc_value,
            'CCC': ccc_value,
            'KendallTau': kendall_tau_value
        })
    return pd.DataFrame(evaluation_results)


# Load data
input_path = os.path.join('RECOLA Ranking Algorithms', 'RECOLA_Intervals_Data', 'ArousalValenceTimeSeries_Quartiles.csv')
df = pd.read_csv(input_path)

feature_columns = [col for col in df.columns if col not in ['participant_id', 'median_arousal', 'median_valence', 'time_window', 'arousal_quartile', 'valence_quartile']]
group_kfold = GroupKFold(n_splits=10)
groups = df['participant_id']

targets = ['arousal_quartile', 'valence_quartile']
for target in targets:
    print(f"\nModeling for {target}:")
    test_accuracies = []
    all_results = [] 

    for train_index, test_index in group_kfold.split(df, groups=groups):
        X_train, X_test = df.iloc[train_index][feature_columns], df.iloc[test_index][feature_columns]
        Y_train, Y_test = df.iloc[train_index][target], df.iloc[test_index][target]
        group_labels_test = groups.iloc[test_index].values
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        tf.keras.backend.clear_session()
        num_features = X_train_scaled.shape[1]
        ord_model = create_ordinal_model(num_classes=11, input_dim=num_features)

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', restore_best_weights=True)
        ord_model.fit(X_train_scaled, Y_train, validation_data=(X_test_scaled, Y_test), epochs=4, batch_size=256, callbacks=[early_stopping], verbose=1)

        predictions = ord_model.predict(X_test_scaled)
        ord_y_pred = np.argmax(predictions[1], axis=1)
        ord_y_pred_score = predictions[0]

        individual_results_df = evaluate_individual_performance(X_test, Y_test, group_labels_test, ord_y_pred_score, target)
        all_results.append(individual_results_df)

    # Combine all results and save to CSV
    output_folder = 'RECOLA Ranking Algorithms/Evaluation/Ordinal Neural Network'
    final_results_df = pd.concat(all_results)
    output_filename = os.path.join(output_folder, f"{target}_evaluation_results.csv")
    final_results_df.to_csv(output_filename, index=False)
    print(f"Results for {target} saved to {output_filename}")


Modeling for arousal_quartile:
Epoch 1/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 2.7277 - val_loss: 2.1185
Epoch 2/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0335 - val_loss: 1.7551
Epoch 3/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6351 - val_loss: 1.4991
Epoch 4/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.3619 - val_loss: 1.3318
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Epoch 1/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.7125 - val_loss: 2.0767
Epoch 2/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.8693 - val_loss: 1.6550
Epoch 3/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.4950 - val_loss: 1.4079
Epoch 4/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2573 - val_loss: 1.2586
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Epoch 1/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 2.4614 - val_loss: 2.6903
Epoch 2/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.9540 - val_loss: 2.1067
Epoch 3/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 